In [1]:
import pandas as pd 
import trueskill as ts

In [2]:
scores_df = pd.read_csv('aug_depressing.csv')

In [3]:
scores_df.shape

(4030216, 3)

In [4]:
scores_df.head()

,left,right,winner
0,50e5f7d4d7c3df413b00056d,50f43537fdc9f065f0002d0e,left
1,50e5f7d4d7c3df413b00056d,50f42c09fdc9f065f000175c,right
2,50e5f7d4d7c3df413b00056d,50f4352cfdc9f065f0002c2b,left
3,50e5f7d4d7c3df413b00056d,50e748e4d7c3df413b0013cb,right
4,50e5f7d4d7c3df413b00056d,50f447d2fdc9f065f0003a81,right


#### Making a list of all unique image ids

In [7]:
all_images = scores_df['left']
all_images = list(all_images)
all_images+= list(scores_df['right'])
len(all_images)

8060432

In [8]:
all_images = list(set(all_images))

In [11]:
len(all_images)

101676

In [12]:
image_ranks={}
for img in all_images:
    image_ranks[img] = ts.Rating()

In [13]:
len(image_ranks)

101676

#### Ranking image pairs

In [14]:
for index, row in scores_df.iterrows():
    l = row['left']; r = row['right']; w = row['winner']
    if w == 'left':
        image_ranks[l], image_ranks[r] = ts.rate_1vs1(image_ranks[l], image_ranks[r])
    elif w == 'right':
        image_ranks[r], image_ranks[l] = ts.rate_1vs1(image_ranks[r], image_ranks[l])
    if index%100000 == 0:
        print(index)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000


In [15]:
len(image_ranks)

101676

#### Writing ranked images to pandas dataframe

In [27]:
column_names = ['image','mean_rank','std_rank']
rank_df = pd.DataFrame(columns = column_names)
index = 0
for key in image_ranks:
    img = key
    mu = image_ranks[key].mu
    sigma = image_ranks[key].sigma
    rank_df.loc[index] = [img, mu, sigma]
    index += 1
    if (index%10000 == 0):
        print(index)
    

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000


In [28]:
rank_df.head()

,image,mean_rank,std_rank
0,50f43514fdc9f065f0002ac5,23.685712,0.897259
1,50f44800fdc9f065f0003beb,26.651815,0.978426
2,51414c52fdc9f049260072e1,28.824266,0.908291
3,51392705818ede129600109f,25.181059,0.890290
4,51414fcbfdc9f04926007ba8,26.777819,0.940551


In [29]:
rank_df.shape

(101676, 3)

In [30]:
rank_df.sort_values('mean_rank',inplace=True)

In [31]:
rank_df.head()

,image,mean_rank,std_rank
31218,513f29a5fdc9f0358700d3e0,11.501665,4.391888
51261,513f3357fdc9f0358700e13f,11.725068,3.997632
7507,513cdf61fdc9f03587002125,11.774503,3.991489
62687,513d56ebfdc9f03587003084,11.789585,4.052904
96872,513d67e5fdc9f035870041bb,11.806957,4.065757


In [32]:
rank_df.to_csv('ranked_depressing_images.csv',index=False)

In [33]:
rank_df.sort_values('mean_rank',inplace=True, ascending=False)
rank_df.head()

,image,mean_rank,std_rank
93271,5140c9f5fdc9f049260026bf,38.624561,3.888543
64695,513d61bafdc9f03587003ea3,38.602080,3.908796
83864,513d5e13fdc9f035870039dd,38.223570,3.902311
46509,513cdc10fdc9f0358700206c,38.186945,4.139627
40002,5140cccdfdc9f04926002ea6,38.146540,4.046895


In [42]:
shuffled_df = rank_df.sample(frac=1)
traindf = shuffled_df.iloc[:81340,:]
testdf = shuffled_df.iloc[81340:,:]

In [45]:
traindf.to_csv('ranked_depressing_train.csv',index=False)
testdf.to_csv('ranked_depressing_test.csv',index=False)

In [44]:
testdf.shape

(20336, 3)

In [46]:
traindf.shape

(81340, 3)